In [2]:
!python --version

Python 3.8.10


In [3]:
! pip install torch seaborn torchvision tensorflow biopython accelerate transformers datasets huggingface_hub evaluate requests pandas sklearn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 31.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.7/199.7 KB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 75.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 37.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 18.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 9.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 73.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 16.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 140.6/140.6 KB 9.9 MB/s eta 0:00:00
  Created wheel for sklearn: filename=sklearn-0.0.post1-p

In [4]:
model_checkpoint = "facebook/esm2_t12_35M_UR50D"

In [5]:
import requests
from io import BytesIO
import pandas as pd
import gc, torch
from accelerate import Accelerator
from Bio import SeqIO 
from Bio.SeqRecord import SeqRecord
import seaborn as sns
gc.collect()
torch.cuda.empty_cache()
accelerator = Accelerator()
device = accelerator.device

For FASTA files from genbank's IPG

In [6]:
#ipg_nottox = []
#for seq_record in SeqIO.parse("ipg_refseq_not_toxins_not_virus_length4to500.fasta", "fasta"):
#  s=str(seq_record.seq)
#  o=seq_record.description.split('[', 1)[1].split(']')[0]
#  i=seq_record.id
#  d=seq_record.description.split(i, 1)[1].split(o)[0]
#  ipg_nottox.append([i,o,s,d])
#ipg_nottox=pd.DataFrame(ipg_nottox,columns=['id','organism','sequence','description'])
#ipg_tox = []
#for seq_record in SeqIO.parse("ipg_refseq_toxins_not_virus_length4to500.fasta", "fasta"):
#  s=str(seq_record.seq)
#  o=seq_record.description.split('[', 1)[1].split(']')[0]
#  i=seq_record.id
#  d=seq_record.description.split(i, 1)[1].split(o)[0]
#  ipg_tox.append([i,o,s,d])
#ipg_tox=pd.DataFrame(ipg_nottox,columns=['id','organism','sequence','description'])

In [7]:
#ipgtoxin_seq=ipg_tox['sequence'].tolist()
#ipgtoxin_labels=[1 for seq in ipgtoxin_seq]
#ipgnottoxin_seq=ipg_nottox['sequence'].tolist()
#ipgnottox_labels=[0 for seq in ipgnottoxin_seq]

TSV files from uniprot/uniref

In [8]:
currentbatchtotal=134500
df0=pd.read_csv('/content/ntref50-4to500#0.tsv',sep='\t')
ntref50=pd.DataFrame(index=range(int(currentbatchtotal-(currentbatchtotal/500)+1)),columns=df0.columns)
ntref50.iloc[:500]=df0
lenlist=[]
for i in range (1,int(currentbatchtotal/500)):
  df=pd.read_csv('/content/ntref50-4to500#'+str((i*500))+'.tsv',sep='\t')
  ntref50.iloc[(i*500)-(i-1):((i+1)*500)-i]=df
tref50=pd.read_csv('/content/tref50.tsv',sep='\t')

FileNotFoundError: ignored

In [ ]:
len(ntref50)

In [ ]:
len(ntref50.dropna())

Export combined data from batch files so it can be imported later without needing to iterate through files again

In [ ]:
data_dir = '/content/drive/My Drive/Colab Notebooks/datasets/'

In [10]:
ntref50.to_csv(data_dir+'ntref50_134500.csv')
tref50.to_csv(data_dir+'tref50.csv')

NameError: ignored

In [11]:
ntref50=pd.read_csv(data_dir+'ntref50_134500.csv')
tref50=pd.read_csv(data_dir+'tref50.csv')

In [12]:
ntref50_sequences=ntref50['Reference sequence'].tolist()
ntref50_labels=[1 for seq in ntref50_sequences]
tref50_sequences=tref50['Reference sequence'].tolist()
tref50_labels=[0 for seq in tref50_sequences]

In [ ]:
sns.set_theme()
sns.barplot([len(ntref50_labels),len(tref50_labels)])

In [18]:
for seq_record in SeqIO.parse(data_dir+"bontx.fasta", "fasta"):
  s=str(seq_record.seq)

In [19]:
print(s)

MKLEINKFNYNDPIDGINVITMRPPRHSDKINKGKGPFKAFQVIKNIWIVPERYNFTNNTNDLNIPSEPIMEADAIYNPNYLNTPSEKDEFLQGVIKVLERIKSKPEGEKLLELISSSIPLPLVSNGALTLSDNETIAYQENNNIVSNLQANLVIYGPGPDIANNATYGLYSTPISNGEGTLSEVSFSPFYLKPFDESYGNYRSLVNIVNKFVKREFAPDPASTLMHELVHVTHNLYGISNRNFYYNFDTGKIETSRQQNSLIFEELLTFGGIDSKAISSLIIKKIIETAKNNYTTLISERLNTVTVENDLLKYIKNKIPVQGRLGNFKLDTAEFEKKLNTILFVLNESNLAQRFSILVRKHYLKERPIDPIYVNILDDNSYSTLEGFNISSQGSNDFQGQLLESSYFEKIESNALRAFIKICPRNGLLYNAIYRNSKNYLNNIDLEDKKTTSKTNVSYPCSLLNGCIEVENKDLFLISNKDSLNDINLSEEKIKPETTVFFKDKLPPQDITLSNYDFTEANSIPSISQQNILERNEELYEPIRNSLFEIKTIYVDKLTTFHFLEAQNIDESIDSSKIRVELTDSVDEALSNPNKVYSPFKNMSNTINSIETGITSTYIFYQWLRSIVKDFSDETGKIDVIDKSSDTLAIVPYIGPLLNIGNDIRHGDFVGAIELAGITALLEYVPEFTIPILVGLEVIGGELAREQVEAIVNNALDKRDQKWAEVYNITKAQWWGTIHLQINTRLAHTYKALSRQANAIKMNMEFQLANYKGNIDDKAKIKNAISETEILLNKSVEQAMKNTEKFMIKLSNSYLTKEMIPKVQDNLKNFDLETKKTLDKFIKEKEDILGTNLSSSLRRKVSIRLNKNIAFDINDIPFSEFDDLINQYKNEIEDYEVLNLGAEDGKIKDLSGTTSDINIGSDIELADGRENKAIKIKGSENSTIKIAMNKYLRFSATDNFSISFWIKHPKPTNLLNNGIEYTLVENFNQRGWKISIQDSK

In [20]:
s in set(ntref50_sequences)

False

In [21]:
s in set(tref50_sequences)

False

In [ ]:
len(ntref50)==int(currentbatchtotal-(currentbatchtotal/500)+1)

In [22]:
sequences=ntref50_sequences+tref50_sequences
labels = tref50_labels+ntref50_labels
len(labels)==len(sequences)

True

In [23]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer, AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

model = AutoModelForSequenceClassification.from_pretrained(model_checkpoint, num_labels=2)
#model = model.to(device)


Some weights of the model checkpoint at facebook/esm2_t12_35M_UR50D were not used when initializing EsmForSequenceClassification: ['lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.decoder.weight']
- This IS expected if you are initializing EsmForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing EsmForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of EsmForSequenceClassification were not initialized from the model checkpoint at facebook/esm2_t12_35M_UR50D and are newly initialized: ['classifier.dense.weight', 'classifier.out_proj.weight', 'classifier.out_proj.bias'

In [24]:
from sklearn.model_selection import train_test_split

train_sequences, test_sequences, train_labels, test_labels = train_test_split(sequences, labels, test_size=0.25, shuffle=True)

In [ ]:
traindf=pd.DataFrame([train_sequences,train_labels],["sequences","labels"])
testdf=pd.DataFrame([test_sequences,test_labels],["sequences","labels"])
traindf.to_csv(data_dir+'ref50_134500_train_seq.csv')
testdf.to_csv(data_dir+'ref50_134500_test_seq.csv')

In [ ]:
print(tokenizer.model_max_length)
max([len(seq) for seq in sequences])

In [ ]:
from datasets import Dataset
train_tokens=tokenizer(train_sequences)
test_tokens=tokenizer(test_sequences)
train_dataset = Dataset.from_dict(train_tokens)
test_dataset = Dataset.from_dict(test_tokens)

In [ ]:
train_dataset = train_dataset.add_column("labels", train_labels)
test_dataset = test_dataset.add_column("labels", test_labels)
# I dont think I need to do this unless I write my own training loop, I think HF uses gpu automatically
#train_dataset=train_dataset.with_format('torch')
#test_dataset=test_dataset.with_format('torch')
#train_dataset.to(device)
#test_dataset.to(device)

A custom callback I was thinking of modifying to help with dynamic plots as the model was trained but am still deciding whether to use it or not.

In [27]:
from transformers import DefaultFlowCallback

In [28]:
class AccuracyCallback(DefaultFlowCallback):
    
    def __init__(self, trainer) -> None:
        super().__init__()
        self._trainer = trainer
    
    def on_epoch_end(self, args, state, control, **kwargs):
        if control.should_evaluate:
            control_copy = deepcopy(control)
            self._trainer.evaluate(eval_dataset=self._trainer.train_dataset, metric_key_prefix="train")
            return control_copy

Arguments for the model, using standard hyperparameters here may change depending on performance. Only using 3 epochs since I think this is what I can get away with before colab kicks me off the GPU.

In [33]:
!pip install wandb -qqq
import wandb
wandb.login()

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 184.0/184.0 KB 23.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.9/178.9 KB 23.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.7/62.7 KB 8.3 MB/s eta 0:00:00


ERROR:wandb.jupyter:Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit: 

··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


True

In [35]:
%env WANDB_PROJECT=esm-fine-tuning-toxins

env: WANDB_PROJECT=esm-fine-tuning-toxins


In [36]:
from evaluate import load
import numpy as np

metric = load("accuracy")
trainingaccuracy=[]
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy=metric.compute(predictions=predictions, references=labels)
    trainingaccuracy.append(accuracy)
    return accuracy

In [37]:
model_name = model_checkpoint.split("/")[-1]
batch_size = 8

args = TrainingArguments(
    f"{model_name}-finetuned-2classtoxin",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=2e-5,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    weight_decay=0.01,
    report_to=['wandb'],
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=False,
)

PyTorch: setting up devices


In [38]:
trainer = Trainer(
    model,
    args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [39]:
#trainer.add_callback(CustomCallback(trainer)) 
train = trainer.train()

/usr/local/lib/python3.8/dist-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num examples = 209576
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 78591
  Number of trainable parameters = 33993843
Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"


wandb: Currently logged in as: chance-challacombe. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy
1,0.322500,0.320154,0.877725
2,0.269400,0.363557,0.884066
3,0.234000,0.475132,0.883923


***** Running Evaluation *****
  Num examples = 69859
  Batch size = 8
Saving model checkpoint to esm2_t12_35M_UR50D-finetuned-2classtoxin/checkpoint-26197
Configuration saved in esm2_t12_35M_UR50D-finetuned-2classtoxin/checkpoint-26197/config.json
Model weights saved in esm2_t12_35M_UR50D-finetuned-2classtoxin/checkpoint-26197/pytorch_model.bin
tokenizer config file saved in esm2_t12_35M_UR50D-finetuned-2classtoxin/checkpoint-26197/tokenizer_config.json
Special tokens file saved in esm2_t12_35M_UR50D-finetuned-2classtoxin/checkpoint-26197/special_tokens_map.json
***** Running Evaluation *****
  Num examples = 69859
  Batch size = 8
Saving model checkpoint to esm2_t12_35M_UR50D-finetuned-2classtoxin/checkpoint-52394
Configuration saved in esm2_t12_35M_UR50D-finetuned-2classtoxin/checkpoint-52394/config.json
Model weights saved in esm2_t12_35M_UR50D-finetuned-2classtoxin/checkpoint-52394/pytorch_model.bin
tokenizer config file saved in esm2_t12_35M_UR50D-finetuned-2classtoxin/checkpoint

In [41]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 69859
  Batch size = 8


{'eval_loss': 0.3635565936565399,
 'eval_accuracy': 0.8840664767603316,
 'eval_runtime': 283.5383,
 'eval_samples_per_second': 246.383,
 'eval_steps_per_second': 30.8,
 'epoch': 3.0}

In [28]:
model_dir = '/content/drive/My Drive/Colab Notebooks/models/'

In [42]:
trainer.save_model(model_dir + 'esm_fine_tuning3epochstrainer')
model.save_pretrained(model_dir + 'esm_fine_tuning3epochsmodel')

Saving model checkpoint to /content/drive/My Drive/Colab Notebooks/models/esm_fine_tuning3epochstrainer
Configuration saved in /content/drive/My Drive/Colab Notebooks/models/esm_fine_tuning3epochstrainer/config.json
Model weights saved in /content/drive/My Drive/Colab Notebooks/models/esm_fine_tuning3epochstrainer/pytorch_model.bin
tokenizer config file saved in /content/drive/My Drive/Colab Notebooks/models/esm_fine_tuning3epochstrainer/tokenizer_config.json
Special tokens file saved in /content/drive/My Drive/Colab Notebooks/models/esm_fine_tuning3epochstrainer/special_tokens_map.json
Configuration saved in /content/drive/My Drive/Colab Notebooks/models/esm_fine_tuning3epochsmodel/config.json
Model weights saved in /content/drive/My Drive/Colab Notebooks/models/esm_fine_tuning3epochsmodel/pytorch_model.bin


eval/accuracy,▁███
eval/loss,▁▃█▃
eval/runtime,▁▃▅█
eval/samples_per_second,█▆▄▁
eval/steps_per_second,█▆▄▁
train/epoch,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇████
train/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
train/learning_rate,████▇▇▇▇▇▆▆▆▆▆▆▅▅▅▅▅▄▄▄▄▄▄▃▃▃▃▃▂▂▂▂▂▂▁▁▁
train/loss,▇█▇▇▇▆▆▆▆▆▆▆▅▅▄▃▄▃▄▄▄▃▅▄▄▄▃▁▂▂▂▂▂▁▃▁▂▂▁▂
train/total_flos,▁
train/train_loss,▁


In [31]:
model = AutoModelForSequenceClassification.from_pretrained(model_dir + 'esm_fine_tuning3epochsmodel', num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

In [38]:
bontx_encoded=tokenizer(s,return_tensors='pt')
bontx_output=model(**bontx_encoded)

In [39]:
from torch import nn
probabilities = nn.functional.softmax(bontx_output.logits, dim=-1)
print(probabilities)

tensor([[0.0655, 0.9345]], grad_fn=<SoftmaxBackward0>)
